In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from data_process import build_vocab, batch_iter, sentence_to_onehot, cal_idf, sentence_to_tfidf
from models import three_layer_net

In [ ]:
train = pd.read_csv('./data/train-5T.txt', delimiter='\t')
test = pd.read_csv('./data/test-1T.txt', delimiter='\t')

In [ ]:
X_train = train.document
Y_train = train.label
X_test = test.document
Y_test = test.label

In [ ]:
max_vocab = 50000
vocab, _, vocab_size = build_vocab(X_train, max_vocab)

# Sentiment Analysis with three layer net using one_hot encoding

In [ ]:
batches = batch_iter(list(zip(X_train, Y_train)), batch_size=64, num_epochs=15)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.reset_default_graph()
sess = tf.Session(config=config)
model = three_layer_net(sess=sess, vocab_size=vocab_size, lr=1e-1)
train_loss = []
train_acc = []
test_loss = []
test_acc = []

for step, batch in enumerate(batches):
    x_train, y_train = zip(*batch)
    x_train = sentence_to_onehot(x_train, vocab)
    acc = model.get_accuracy(x_train, y_train)
    l, _ = model.train(x_train, y_train)
    train_loss.append(l)
    train_acc.append(acc)
    
    if step % 100 == 0:
        test_batches = batch_iter(list(zip(X_test, Y_test)), batch_size=64, num_epochs=1)
        for test_batch in test_batches:
            x_test, y_test = zip(*test_batch)
            x_test = sentence_to_onehot(x_test, vocab)
            t_acc = model.get_accuracy(x_test, y_test)
            t_loss = model.get_loss(x_test, y_test)
            test_loss.append(t_loss)
            test_acc.append(t_acc)
        print('batch:', '%04d' % step, '\ntrain loss:', '%.5f' % np.mean(train_loss), 
              '\ttest loss:', '%.5f' % np.mean(test_loss))
        print('train accuracy:', '%.3f' % np.mean(train_acc), '\ttest accuracy:', 
              '%.3f' % np.mean(test_acc), '\n')
        train_loss = []
        train_acc = []
        test_loss = []
        test_acc = []

# Sentiment Analysis with three layer net using tfidf encoding

In [ ]:
IDF = cal_idf(X_train, vocab)

In [ ]:
batches = batch_iter(list(zip(X_train, Y_train)), batch_size=64, num_epochs=15)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.reset_default_graph()
sess = tf.Session(config=config)
model = three_layer_net(sess=sess, vocab_size=vocab_size, lr=1e-1)
train_loss = []
train_acc = []
test_loss = []
test_acc = []

for step, batch in enumerate(batches):
    x_train, y_train = zip(*batch)
    x_train = sentence_to_tfidf(x_train, vocab, IDF)
    acc = model.get_accuracy(x_train, y_train)
    l, _ = model.train(x_train, y_train)
    train_loss.append(l)
    train_acc.append(acc)
    
    if step % 100 == 0:
        test_batches = batch_iter(list(zip(X_test, Y_test)), batch_size=64, num_epochs=1)
        for test_batch in test_batches:
            x_test, y_test = zip(*test_batch)
            x_test = sentence_to_tfidf(x_test, vocab, IDF)
            t_acc = model.get_accuracy(x_test, y_test)
            t_loss = model.get_loss(x_test, y_test)
            test_loss.append(t_loss)
            test_acc.append(t_acc)
        print('batch:', '%04d' % step, '\ntrain loss:', '%.5f' % np.mean(train_loss), 
              '\ttest loss:', '%.5f' % np.mean(test_loss))
        print('train accuracy:', '%.3f' % np.mean(train_acc), '\ttest accuracy:', 
              '%.3f' % np.mean(test_acc), '\n')
        train_loss = []
        train_acc = []
        test_loss = []
        test_acc = []